### Clean data


In [ ]:
import sys
sys.path.append("../..")

In [ ]:
import pandas as pd
from config.connect_db import ConnectDataBase
from sqlalchemy import text as sql_text
from ydata_profiling import ProfileReport

db = ConnectDataBase()
engine = db.engine()

### Query


In [ ]:
def query_table_dtypes(params: str) -> str:
    return f"""       
                SELECT column_name, data_type
                FROM information_schema.columns
                WHERE table_name = '{params}'
                ORDER BY ordinal_position; 
            """


df_calendar_tdtypes = pd.read_sql(sql=sql_text(
    query_table_dtypes("Calendar_Bronze")), con=engine.connect())
df_listing_tdtypes = pd.read_sql(sql=sql_text(
    query_table_dtypes("Listing_Bronze")), con=engine.connect())
df_reviews_tdtypes = pd.read_sql(sql=sql_text(
    query_table_dtypes("Reviews_Bronze")), con=engine.connect())

In [ ]:
def query_table(params: str) -> str:
    return f"""
          SELECT * 
          FROM public."{params}";
         """

### Results from bank tables


In [ ]:
df_calendar_silver = pd.read_sql(sql=sql_text(
    query_table("Calendar_Bronze")), con=engine.connect())

In [ ]:
df_listing_silver = pd.read_sql(sql=sql_text(
    query_table("Listing_Bronze")), con=engine.connect())

In [ ]:
df_reviews_silver = pd.read_sql(sql=sql_text(
    query_table("Reviews_Bronze")), con=engine.connect())

### Copy Dataframe origin


In [ ]:
df_listing_silver_cln = df_listing_silver.copy()
df_reviews_silver_cln = df_reviews_silver.copy()
df_calendar_silver_cln = df_calendar_silver.copy()

### DTypes Dataframe


In [ ]:
dtypes = pd.DataFrame(df_listing_silver_cln.dtypes)
df_half_one = dtypes.head(45)
df_half_two = dtypes.iloc[45:]
print("Primeira parte")
display(df_half_one)
print("Segunda parte")
display(df_half_two)

In [ ]:
columns_df = pd.DataFrame(df_listing_silver_cln.select_dtypes(
    include=['object']).dtypes, columns=['name']).reset_index()

columns_df.columns = ['column_name', 'data_type']

mask = df_listing_tdtypes['column_name'].isin(columns_df['column_name'])
filtered_df = df_listing_tdtypes[mask]
mask_text = filtered_df['data_type'] != 'text'
filtered_df = filtered_df.loc[mask_text]['column_name'].tolist()

list_float = ['price']

for column in list_float:
    df_listing_silver_cln[column] = df_listing_silver_cln[column].str.replace(
        '$', '').str.replace(',', '').astype(float)

for column in filtered_df:
    df_listing_silver_cln[column] = df_listing_silver_cln[column].astype(
        "float")


dtypes = pd.DataFrame(df_listing_silver_cln.dtypes)
df_half_one = dtypes.head(45)
df_half_two = dtypes.iloc[45:]
print("Primeira parte")
display(df_half_one)
print("Segunda parte")
display(df_half_two)

In [ ]:
dtypes = pd.DataFrame(df_reviews_silver_cln.dtypes)

list_datetime = ['date']

for column in list_datetime:
    df_reviews_silver_cln[column] = pd.to_datetime(
        df_reviews_silver_cln[column], format='%Y-%m-%d')

dtypes = pd.DataFrame(df_reviews_silver_cln.dtypes)
display(dtypes)

In [ ]:
dtypes = pd.DataFrame(df_calendar_silver_cln.dtypes)

list_datetime = ['date']
list_float = ['price', 'adjusted_price']

for column in list_datetime:
    df_calendar_silver_cln[column] = pd.to_datetime(
        df_calendar_silver_cln[column], format='%Y-%m-%d')

for column in list_float:
    df_calendar_silver_cln[column] = df_calendar_silver_cln[column].str.replace(
        '$', '').str.replace(',', '').astype(float)

dtypes = pd.DataFrame(df_calendar_silver_cln.dtypes)
display(dtypes)

### Replace columns


In [ ]:
df_listing_silver_cln.columns = df_listing_silver_cln.columns.str.replace(
    r'\W', '_', regex=True)
df_reviews_silver_cln.columns = df_reviews_silver_cln.columns.str.replace(
    r'\W', '_', regex=True)
df_calendar_silver_cln.columns = df_calendar_silver_cln.columns.str.replace(
    r'\W', '_', regex=True)

### Handling null values


In [ ]:
def check_missing(df):
    res_missing = df.isna().sum()
    res_missing = (res_missing/len(df))*100
    return res_missing[res_missing > 0]

In [ ]:
print("Valores null antes")
display(pd.DataFrame(check_missing(
    df_listing_silver_cln).sort_values(ascending=False)))

colums_filnan = pd.DataFrame(check_missing(
    df_listing_silver_cln).sort_values(ascending=False), columns=['name']).reset_index()
colums_filnan.columns = ['column_name', 'percentage']

columns_listing_silver_cln = colums_filnan['column_name'].to_list()

df_listing_silver_cln[columns_listing_silver_cln] = df_listing_silver_cln[columns_listing_silver_cln].fillna(
    "no_info")

print("Valores null depois")
display(pd.DataFrame(check_missing(
    df_listing_silver_cln).sort_values(ascending=False)))

In [ ]:
print("Valores null antes")
display(pd.DataFrame(check_missing(
    df_reviews_silver_cln).sort_values(ascending=False)))

colums_filnan = pd.DataFrame(check_missing(
    df_reviews_silver_cln).sort_values(ascending=False), columns=['name']).reset_index()
colums_filnan.columns = ['column_name', 'percentage']

columns_reviews_silver_cln = colums_filnan['column_name'].to_list()

df_reviews_silver_cln[columns_reviews_silver_cln] = df_reviews_silver_cln[columns_reviews_silver_cln].fillna(
    "no_info")

print("Valores null depois")
display(pd.DataFrame(check_missing(
    df_reviews_silver_cln).sort_values(ascending=False)))

In [ ]:
print("Valores null antes")
display(pd.DataFrame(check_missing(
    df_calendar_silver_cln).sort_values(ascending=False)))

colums_filnan = pd.DataFrame(check_missing(
    df_calendar_silver_cln).sort_values(ascending=False), columns=['name']).reset_index()
colums_filnan.columns = ['column_name', 'percentage']

columns_calendar_silver_cln = colums_filnan['column_name'].to_list()

df_calendar_silver_cln[columns_calendar_silver_cln] = df_calendar_silver_cln[columns_calendar_silver_cln].fillna(
    "no_info")

print("Valores null depois")
display(pd.DataFrame(check_missing(
    df_calendar_silver_cln).sort_values(ascending=False)))

### Drop duplicateds


In [ ]:
print("N. de linhas da tabela listing antes de remover duplicadas:",
      len(df_listing_silver_cln))
df_listing_silver_cln = df_listing_silver_cln.drop_duplicates()
print("N. de linhas da tabela listing depois de remover duplicadas:",
      len(df_listing_silver_cln))

In [ ]:
print("N. de linhas da tabela reviews antes de remover duplicadas:",
      len(df_reviews_silver_cln))
df_reviews_silver_cln = df_reviews_silver_cln.drop_duplicates()
print("N. de linhas da tabela reviews depois de remover duplicadas:",
      len(df_reviews_silver_cln))

In [ ]:
print("N. de linhas da tabela calendar antes de remover duplicadas:",
      len(df_calendar_silver_cln))
df_calendar_silver_cln = df_calendar_silver_cln.drop_duplicates()
print("N. de linhas da tabela calendar depois de remover duplicadas:",
      len(df_calendar_silver_cln))

### Removing special characters


In [ ]:
pattern = r'<[^>]+>'
df_listing_silver_cln['description'] = df_listing_silver_cln['description'].str.replace(
    pattern, '', regex=True)


df_listing_silver_cln['description'] = df_listing_silver_cln['description'].str.strip(
).str.lower().str.replace('[^a-z\s]', '', regex=True)

### Eliminating extremely unbalanced speakers


In [ ]:
df_listing_silver_cln['maximum_nights'].value_counts(normalize=True)

In [ ]:
df_listing_silver_cln['maximum_nights'].value_counts(normalize=True).values[0]

In [ ]:
list_imbalance = []
limit = 0.98
for col in df_listing_silver_cln.columns:
    perc = df_listing_silver_cln[col].value_counts(normalize=True).values[0]
    if perc > limit:
        list_imbalance.append(col)
        print(col, perc)

In [ ]:
df_listing_silver_cln = df_listing_silver_cln.drop(list_imbalance, axis=1)

In [ ]:
list_imbalance = []
limit = 0.98
for col in df_listing_silver_cln.columns:
    perc = df_listing_silver_cln[col].value_counts(normalize=True).values[0]
    if perc > limit:
        list_imbalance.append(col)
        print(col, perc)

### Checking data accuracy


In [ ]:
df_listing_silver_cln = df_listing_silver_cln[df_listing_silver_cln['price'] > 0]

In [ ]:
df_listing_silver_cln.describe()

### Outliers


In [ ]:
# valor do quantil de 2% a esquerda
df_listing_silver_cln[['price', 'maximum_nights']].quantile(.02)

In [ ]:
# valor do quantil de 2% a direita
df_listing_silver_cln[['price', 'maximum_nights']].quantile(.98)

In [ ]:
print("Quantidade de linhas antes de eliminar os outliers:",
      len(df_listing_silver_cln))
# colunas para considerar a eliminação de outliers
list_quantile = ['price', 'maximum_nights']
df_aux = df_listing_silver_cln.copy()
for col in list_quantile:
    low_limit = df_aux[col].quantile(.02)  # valor do quantil de 2% a esquerda
    high_limit = df_aux[col].quantile(.98)  # valor do quantil de 2% a direita
    df_listing_silver_cln = df_listing_silver_cln[(df_listing_silver_cln[col] > low_limit) & (
        df_listing_silver_cln[col] < high_limit)]  # filtra a partir do quantil

print("Quantidade de linhas depois de eliminar os outliers:",
      len(df_listing_silver_cln))

In [ ]:
df_listing_silver_cln.describe()

### Saving dataframe in silver table


In [ ]:
df_listing_silver_cln.to_sql(
    'Listing_Silver', engine, if_exists='replace', index=False)

In [ ]:
df_reviews_silver_cln.to_sql(
    'Reviews_Silver', engine, if_exists='replace', index=False)

In [ ]:
df_calendar_silver_cln.to_sql(
    'Calendar_Silver', engine, if_exists='replace', index=False)

Reports profiles


In [ ]:
profile = ProfileReport(
    df_listing_silver_cln, title="Pandas Profiling Report")

profile.to_file(f"reports/Listing_Silver.html")

In [ ]:
profile = ProfileReport(
    df_reviews_silver_cln, title="Pandas Profiling Report")

profile.to_file(f"reports/Reviews_Silver.html")

In [ ]:
profile = ProfileReport(
    df_calendar_silver_cln, title="Pandas Profiling Report")

profile.to_file(f"reports/Calendar_Silver.html")